### testing the XGBoost Diabetes example to transpile
##### used conda env giza from ll laptop

## Create and Train an LR Model
### We'll start by creating a simple XGBoost model using Scikit-Learn and train it on ship ETA dataset

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from giza.datasets import DatasetsLoader
from giza.agents import GizaAgent
from giza.zkcook import serialize_model

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns



In [4]:
data  = pd.read_csv('Container-short-V5-A-1.csv')
data.head(4)


FileNotFoundError: [Errno 2] No such file or directory: 'Container-short-V5-A-1.csv'

In [3]:
data.shape

(1186, 8)

In [4]:
#X, y = data.data, data.target

# Drop rows with missing values
df_cleaned = data.dropna(subset=['X_NAUT_DIST_KM'])

# Prepare your data
X = df_cleaned[['X_NAUT_DIST_KM']]
y = df_cleaned['Y_POD_ETA_DAYS']


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [6]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("linear_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## Save the model
### Save the model in Json format

## Transpile your model to Orion Cairo
### We will use Giza-CLI to transpile our saved model to Orion Cairo.


$ giza transpile 

## Deploy an inference endpoint
### Now that our model is transpiled to Cairo we can deploy an endpoint to run verifiable inferences. We will use Giza CLI again to run and deploy an endpoint. Ensure to replace model-id and version-id with your ids provided during transpilation.


giza endpoints deploy --model-id 665 --version-id 1

$ giza endpoints deploy --model-id XXX --version-id X


## Run a verifiable inference
##### To streamline verifiable inference, you might consider using the endpoint URL obtained after transpilation. However, this approach requires manual serialization of the input for the Cairo program and handling the deserialization process. To make this process more user-friendly and keep you within a Python environment, we've introduced a Python SDK designed to facilitate the creation of ML workflows and execution of verifiable predictions. When you initiate a prediction, our system automatically retrieves the endpoint URL you deployed earlier, converts your input into Cairo-compatible format, executes the prediction, and then converts the output back into a numpy object. 

In [9]:
from giza.agents.model import GizaModel

MODEL_ID = 699  # Update with your model ID
VERSION_ID = 6  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True
    )

    return result, proof_id

def execution():
    # The input data type should match the model's expected input
    input = np.array([[7200.1]]).astype(np.float32)

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(
        f"Predicted value for input {input.flatten()[0]} is {result[0].flatten()[0]}")

    return result, proof_id


execution()

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 7200.10009765625 is 26.45867919921875


(array([[26.4586792]]), '7243d905ff824fc996adbd99f720cd82')

## Download the proof
#### Initiating a verifiable inference sets off a proving job on our server, sparing you the complexities of installing and configuring the prover yourself. Upon completion, you can download your proof.

First, let's check the status of the proving job to ensure that it has been completed.

$ giza endpoints get-proof --endpoint-id XXX --proof-id "xxxxxxxxxxxxxxxxxxxxxxxxxx"
[giza][2024-05-30 00:40:39.691] Getting proof from endpoint xxx
{
  "id": 967,
  "job_id": 1121,
  "metrics": {
    "proving_time": 17.249508
  },
  "created_date": "2024-05-30T07:33:12.532659"
}

Once the proof is ready, you can download it.

$ giza endpoints download-proof --endpoint-id 234 --proof-id "10c164e6c2364ab6b5491702127979a6" --output-path zk_xgboost.proof
[giza][2024-05-30 00:51:52.048] Getting proof from endpoint 234 ✅
[giza][2024-05-30 00:51:53.800] Proof downloaded to zk_xgboost.proof ✅
(giza3)

## Verify the proof
#### Finally, you can verify the proof.

$ giza verify --proof-id 967
[giza][2024-05-30 00:56:05.847] Verifying proof...
[giza][2024-05-30 00:56:07.140] Verification result: True
[giza][2024-05-30 00:56:07.145] Verification time: 0.454667226
(giza3)